In [1]:
import pandas as pd
import numpy as np
import re

# Pre-processamento
from nltk import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import cross_val_score, GridSearchCV, train_test_split

# Classificadores
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC 
from sklearn.linear_model import LogisticRegression

from warnings import filterwarnings

# Ignorando warnings de deprecation
filterwarnings(action='ignore')

In [2]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [3]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [4]:
data.shape

(2606, 2)

In [5]:
# Removendo o atributo ID (irrelevante)
ids = data['ID']
data.drop(['ID'], axis=1, inplace=True)

In [6]:
# Inicializando atributos target com 0.0
data['SMARTPHONE'] = np.zeros((len(data)))

# Parte 1: Anotação dos dados
Objetivando a aplicação de um algoritmo supervisionado, foram classificados manualmente os dados entre 1.0 (Smartphone) e 0.0 (Não-Smartphone) e depois filtrados os possíveis casos indesejáveis. Tal tarefa é relativamente simples, pois os intervalos onde há smartphones são facilmente identificáveis. Caso os dados estivessem muito embaralhados e/ou o conjunto de dados fosse muito grande, outras técnicas poderiam ser usadas (i.e. clustering, regex).

In [7]:
# Manualmente selecionando os intervalos onde existem smartphones
data['SMARTPHONE'].iloc[6:39] = 1.0
data['SMARTPHONE'].iloc[2035:] = 1.0

# Nota: Há alguns smartwatches nesse intervalo.
#       Além disso, pode ser que haja algum smartphone que não esteja nesse intervalo.
#       Esses casos serão tratados a seguir

In [8]:
# Casos de Smartwatches classificados como 1.0
for i, example in data[data['SMARTPHONE'] == 1.0]['TITLE'].iteritems():
    if re.findall('watch|rel[ó|o]gio', example.lower()):
        print(i, example)
        data['SMARTPHONE'].iloc[i] = 0.0

2207 Apple Watch Série 3 42mm Cores E Edição  Nike+ Em Estoque
2227 Apple Watch Series 3 42mm Nike + Gps Prova D'água Lacrado
2244 Apple Watch Series 3 42mm Gps Garantia Apple + Brinde
2253 Relógio Samsung Gear S3 Sm-r770 Classic Wifi Bluetooth
2288 Apple Watch Series 3  42mm Nike + Gps Prova D'água Lançameto
2289 Apple Watch Series 3 42mm Nike +  Gps Prova D'água
2298 Apple Watch Series 3 38mm Nike + Gps Prova D'água
2301 Apple Watch Nike+ Séries 3 42mm Lacrado
2302 Apple Watch Nike+ Séries 3 38mm Lacrado Novo
2303 Apple Watch Nike+ Séries 3 38mm Lacrado
2309 Apple Watch Series 3 38mm Nike + Gps Prova D'água Lacrado
2314 Apple Watch Série 3 42mm Gps + 2 Pulseiras Extras
2333 Relógio Samsung S3 Frontier + 2 Pulseiras Extras
2345 Apple Watch Series 3 42mm Lacrado + Brinde
2353 Apple Watch Nike+ Séries 3 42mm Lacrado + Envio Imediato
2380 Apple Watch Series 3 38mm Lacrado Nf + Brinde
2417 Smartwatch Samsung Gear S3 Frontier 1.3 Bluetooth - Original
2597 Relógio Smartwatch Samsung Gear Sm

In [9]:
# Casos de smartphones classificados como 0.0
for i, example in data[data['SMARTPHONE'] == 0.0]['TITLE'].iteritems():
    if re.findall('^smartphone|^celular', example.lower()):
        print(i, example)
        
# Somente 2 casos
data['SMARTPHONE'].iloc[1892:1894] = 1.0

# Nota: Foram encontrados alguns celulares, mas que não são smartphones. Estes foram desconsiderados.

548 Celular Multilaser Up- Branco/Rosa
775 Celular Blu Samba Jr Q53 Prata Dual Chip Silver
1886 Celular Up 2chip Quad Cam Mp3/4 Fm Branco/Rosa
1892 Smartphone Samsung Galaxy J2 Prime TV Dourado com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz
1893 Smartphone Samsung Galaxy J2 Prime TV Preto com 8GB, Dual chip, Tela 5", TV Digital, 4G, Câmera 8MP, Android 6.0 e Processador Quad Core de 1.4 Ghz


In [10]:
data['SMARTPHONE'].value_counts()

0.0    2018
1.0     588
Name: SMARTPHONE, dtype: int64

In [11]:
data['SMARTPHONE'].mean()
# Classes estão desbalanceadas

0.2256331542594014

# Parte 2: Pré-processamento
Nessa etapa serão feitos processos para normalizar os dados. Serão removidas stopwords, todas as palavras serão transformadas em representações de letras minúsculas e então será feita tokenização e bag of words.

In [12]:
# Adicionando pontuações às stopwords em português
pontuacoes = ['-', '.', ',', '(', ')']
pt_stopwords = stopwords.words('portuguese')

for pontuacao in pontuacoes:
    pt_stopwords.append(pontuacao)

In [13]:
# CountVectorizer remove os stopwords, tokeniza e obtem bag of words
vect = CountVectorizer(stop_words=pt_stopwords)
vect = CountVectorizer().fit(data['TITLE'])

data_vect = vect.transform(data['TITLE'])

In [14]:
data_vect

<2606x4165 sparse matrix of type '<class 'numpy.int64'>'
	with 21947 stored elements in Compressed Sparse Row format>

# Parte 3: Seleção do Modelo e Validação
Será utilizado Grid Search para encontrar os melhores parâmetros para cada classificador. O GS tem 5 folds de cross validation, método de avaliação f1 (pois as classes estão desbalanceadas) e é feita paralelização em 4 threads. Os classificadores escolhidos para a tarefa foram:

### SVM:
- Funciona bem em dados com alta dimensão e poucos exemplos (dataset está com 2606 linhas e 4165 colunas);
- Variando parâmetros C e gamma.

### Regressão Logística:
- Modelo simples e eficiente;
- Variando penalidades l1 e l2 e o parâmetro C.

### Bernoulli:
- Naive Bayes de representação binária;
- Variando constante de suavização de laplace.

### Multinomial:
- Naive Bayes para representação de múltiplas ocorrências;
- Variando constante de suavização de laplace.

In [15]:
# Definindo os parâmetros a serem selecionados para cada classificador
SVM = {'name': 'svm',
       'classifier': SVC(),
       'parameters': {'kernel': ['linear'],
       'C': [0.1, 1.0, 5, 10, 15],
       'gamma': [0.01, 0.1, 1.0, 5.0, 10],}}

Logistic = {'name': 'logistic_regression',
       'classifier': LogisticRegression(),
       'parameters': {'penalty': ['l1', 'l2'],
                      'C': [0.01, 0.1, 1.0, 5.0, 10],
                      'solver': ['liblinear']}}

Bernoulli = {'name': 'bernoulli_naive_bayes',
             'classifier': BernoulliNB(),
             'parameters': {'alpha': [1.0, 2.0, 5.0]}}
            
Multinomial = {'name': 'multinomial_naive_bayes',
               'classifier': MultinomialNB(),
               'parameters': {'alpha': [1.0, 2.0, 5.0]}}

classificadores = [SVM, Logistic, Bernoulli, Multinomial]

In [16]:
# Encontra os melhores parâmetros para cada classificador

modelos = {}
for classificador in classificadores:
    gs = GridSearchCV(estimator=classificador['classifier'],
                                param_grid=classificador['parameters'],
                                scoring='f1',
                                verbose=0,
                                cv=5,
                                n_jobs=4)

    gs.fit(data_vect, data['SMARTPHONE'])
    modelos[classificador['name']] = gs.best_estimator_

In [17]:
# Avaliação usando cross validation score
resultados = {}
for nome, modelo in modelos.items():
    resultados[nome] = np.mean(cross_val_score(modelo,
                                            data_vect,
                                            data['SMARTPHONE'],
                                            cv=5,
                                            scoring='f1'))
    
for nome, resultado in resultados.items():
    print(nome+':', resultado)
    
# Regressor Logístico > SVM > BernoulliNB > MultinomialNB

svm: 0.9820169917387778
logistic_regression: 0.9835542996094679
bernoulli_naive_bayes: 0.9485674493873768
multinomial_naive_bayes: 0.9203615982819263


In [18]:
# 100 principais features associadas a smartphones
feature_names = np.array(vect.get_feature_names())
coefs_sorted = modelos['logistic_regression'].coef_[0].argsort()
feature_names[coefs_sorted][-100:]

array(['1sim', '6gb', 'mini', 'xt1676', 'j5', 'g6', 'indigo', 'j810m',
       'orig', '20mp', 'câmera', 'g570m', 'j8', 'g5', 'duro', 'termica',
       'queda', 'na', 'xperia', 'pronta', 'leitor', 'grátis', '3gb',
       'entrega', 'no', 'grafite', 'pocket', 'a1', '2017', 'digital',
       'test', 'nuevo', 'ofertar', 'item', 'sim', 'frete', 'processador',
       '32', 's5', 'duos', 'promoção', '4gb', 'ano', 'android', 'tv',
       'j400m', 'j4', 's60', 'caterpillar', 'cat', 'k10', 'play', 'redmi',
       'e5', 'global', 'chip', 'apple', 'samsung', 'desbloqueado',
       '128gb', 'lg', 'xt1726', 'mi', 'fiscal', 'ram', '8mp', 'nf',
       'garantia', 'prime', 'nota', '8gb', 'moto', '3g', 'gb', 'j7',
       'ferrari', 'anatel', 'novo', 'vitrine', 'dual', 'nextel',
       'dourado', '4g', 'lacrado', 'g800h', '16gb', 'tela', 'plus',
       'original', 'xiaomi', '256gb', 'celular', 's9', 'galaxy',
       'motorola', 'smartphone', 'i867', 'iphone', '32gb', '64gb'],
      dtype='<U23')

In [19]:
# 100 principais features associadas a não-smartphones
feature_names[coefs_sorted][:100]

array(['capa', 'bumper', 'bracadeira', 'para', 'película', 'vidro', 'tab',
       'rosa', 'kit', 'watch', 'tablet', 'tpu', 'p355m', 'azul',
       'transparente', 'vermelho', 'pc', 'de', 'bumber', 's3', '5mp',
       'fonte', 'series', '38mm', 'pelicula', 'em', 'protetora',
       'shampoo', 'preta', 'único', 'capapelicula', 'pingente', '4s',
       'ero', 'laço', 'cabo', 'papel', 'do', 'silver', 'nike', 's6102',
       's4', 'samba', 'jr', 'q53', 'carregador', 'flip', 'ml', 'blu',
       'cm', 'ciabrink', 'premium', 'dados', 'couro', 'séries', '42mm',
       'core', 'pulseira', 'amarelo', 'ghz', 'violão', 'sm', 'infantil',
       'temperado', 'litocart', 'branco', 'mirror', 'usb', 'black',
       'adesivo', 'madeira', 'espelhamento', 'au906', 'cast', 'esportiva',
       'batom', 'verde', 'micro', 'vgn', 'veicular', 'adaptador',
       'braçadeira', 'idea', 'gps', '5c', 'tracta', 'whey', 'exclusiva',
       'personalizada', 'branca', 'ddr3', 'com', 'suporte', 'coração',
       'botão',

# Parte 4: Classificação
Aplicação do melhor modelo encontrado e salvamento do resultado.

In [20]:
data = pd.read_csv('data/data_estag_ds.tsv', sep='\t')

In [21]:
data.head(10)

,ID,TITLE
0,1041354,Acessório T - Jean Bag For Girls para DS Lite
1,1041782,Carrinho de Bebê Berço-Passeio - Pegasus Pink ...
2,1041834,Carrinho de Bebê para Gêmeos Berço-Passeio - T...
3,1042568,Car Center - Calesita
4,1042584,Donka Trem com Som - Calesita
5,1043696,Interfone HDL AZ-01 Branco
6,1940486,Smartphone LG Optimus L7 II Dual P716 Preto co...
7,2277815,Smartphone Samsung Galaxy S4 Mini Duos Preto c...
8,2961097,Smartphone Samsung Galaxy S5 SM-G900M Branco ...
9,2961099,Smartphone Samsung Galaxy S5 SM-G900M Dourado...


In [22]:
vect = CountVectorizer(stop_words=pt_stopwords)
X = CountVectorizer().fit_transform(data['TITLE'])
y = modelos['logistic_regression'].predict(X)

In [23]:
modelos['logistic_regression']

LogisticRegression(C=10, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [24]:
data['SMARTPHONE'] = y

# SEGUNDA FASE

In [25]:
import nltk
from fuzzywuzzy import process

In [26]:
birdie_data = pd.read_csv('data/resultados.tsv', sep='\t')
submarino_data = pd.read_csv('data/submarino_smartphones.tsv', sep='\t')

In [27]:
submarino_data.head()

,url,smartphone
0,https://www.submarino.com.br/categoria/celular...,Smartphone Motorola One Vision 128GB Dual Chip...
1,https://www.submarino.com.br/categoria/celular...,"Iphone 8 Plus 64GB Ouro Tela 5.5"" iOS 12 4G C..."
2,https://www.submarino.com.br/categoria/celular...,iPhone 8 Plus 64gb Silver Tela 5.5” iOS 12 4G ...
3,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy S10e 128GB Dual Chip...
4,https://www.submarino.com.br/categoria/celular...,Smartphone Samsung Galaxy Note 9 128GB Nano Ch...


In [28]:
vect = CountVectorizer().fit(data['TITLE'])
X = vect.transform(submarino_data['smartphone'])
y = modelos['logistic_regression'].predict(X)
submarino_data['SMARTPHONE'] = y

In [29]:
submarino_data['SMARTPHONE'].value_counts()

1.0    5246
0.0    1479
Name: SMARTPHONE, dtype: int64

In [30]:
submarino_data[submarino_data['SMARTPHONE'] == 0].head(20)

,url,smartphone,SMARTPHONE
116,https://www.submarino.com.br/categoria/celular...,Usado: Iphone 7 Apple 128gb Prata,0.0
129,https://www.submarino.com.br/categoria/celular...,Usado: Iphone 6 16gb Cinza Espacial Muito Bom ...,0.0
130,https://www.submarino.com.br/categoria/celular...,Smartwatch Samsung Galaxy Watch Active Sm-r500...,0.0
135,https://www.submarino.com.br/categoria/celular...,Moto G6 Plus Edição Limitada,0.0
141,https://www.submarino.com.br/categoria/celular...,Usado: Iphone 7 128gb Preto Brilhante Bom - Tr...,0.0
164,https://www.submarino.com.br/categoria/celular...,Smartwatch Samsung Galaxy Watch Active Sm-r500...,0.0
168,https://www.submarino.com.br/categoria/celular...,Zenfone 4 4GB Ram - Preto,0.0
193,https://www.submarino.com.br/categoria/celular...,Usado: Iphone 6s Apple 128gb Prata,0.0
196,https://www.submarino.com.br/categoria/celular...,Usado: Samsung Galaxy Note 9 Preto Muito Bom -...,0.0
203,https://www.submarino.com.br/categoria/celular...,Seminovo: iPhone 7 128GB Preto Matte Usado,0.0


In [31]:
submarino_data = pd.DataFrame(submarino_data['smartphone'][submarino_data['SMARTPHONE'] == 1])
submarino_data.columns = ['TITLE']

birdie_data = pd.DataFrame(birdie_data['TITLE'][birdie_data['SMARTPHONE'] == 1], columns=['TITLE'])

In [32]:
print('tamanho (submarino):', len(submarino_data['TITLE']))
print('qtd de smartphones unicos:', submarino_data['TITLE'].nunique())

submarino_data.drop_duplicates(inplace=True)

tamanho (submarino): 5246
qtd de smartphones unicos: 4061


In [33]:
print('tamanho (birdie):', len(birdie_data['TITLE']))
print('qtd de smartphones unicos:', birdie_data['TITLE'].nunique())

tamanho (birdie): 588
qtd de smartphones unicos: 546


In [34]:
submarino_data['TITLE'] = submarino_data['TITLE'].apply(lambda x: x.lower())

In [35]:
submarino_data.head()

,TITLE
0,smartphone motorola one vision 128gb dual chip...
1,"iphone 8 plus 64gb ouro tela 5.5"" ios 12 4g c..."
2,iphone 8 plus 64gb silver tela 5.5” ios 12 4g ...
3,smartphone samsung galaxy s10e 128gb dual chip...
4,smartphone samsung galaxy note 9 128gb nano ch...


In [36]:
birdie_data['TITLE'] = birdie_data['TITLE'].apply(lambda x: x.lower())

In [37]:
birdie_data.head()

,TITLE
6,smartphone lg optimus l7 ii dual p716 preto co...
7,smartphone samsung galaxy s4 mini duos preto c...
8,smartphone samsung galaxy s5 sm-g900m branco ...
9,smartphone samsung galaxy s5 sm-g900m dourado...
10,smartphone lg l90 dual d410 preto com tela de ...


In [38]:
#re_marcas = ('samsung|lg|motorola|asus|apple|multilaser|sony|xiaomi|' + 
#             'alcatel|positivo|microsoft|lenovo|huawei|caterpillar')

re_qtds_memoria = '[ ,\.\\/]\d{1}gb[ ,\.\\/]'

re_qtds_armazenamento = '[ ,\.\\/]\d{2,3}gb[ ,\.\\/]'

#re_cor = 'azul|amarelo|vermelho|branco|preto|dourado|prata|violeta|grafite|black piano'

# re_dual_chip = 'dual chip|dual sim'

#re_tam_tela = '[4-7][\.,]\d'

re_modelos = ('galaxy pocket plus?|galaxy core plus|galaxy \w\d{1,2} prime|galaxy \w\d{1,2} mini|' +  # Samsung
              'galaxy \w\d{1,2} edge|galaxy \w\d{1,2}\w?\+?|galaxy .* duos|galaxy note \d|' + # Samsung
              'moto \w\d? plus|moto \w\d?|nextel \w\d{1,3}|motorola \w plus|' + # Motorola
              'redmi note \d lite|redmi note \d|mi mix \d\w|mi \w?\d|' + # Xiaomi
              'mi \w?\d lite|pocophone \w\d{1,2}|' + # Xiaomi
              'iphone \d plus|iphone \d\w plus|iphone \d|iphone \d\w|iphone x[sr] plus|iphone x[sr]|iphone x|' + # Apple
              'optimus \w\d{1,2}|\w\d{1,2} lite|k\d{1,2}|q\d\+ platinum|' + # LG
              'zenfone \d|zenfone go|zenfone i+|' + # Asus
              'cat \w\d{1,2}|' + # Caterpillar
              'xperia \w{1,2}\d plus \w*\d{1,4}|xperia \w{1,2}\d \w*\d{1,4}|xperia \w{1,2}\d|' + # Sony
              'ms\d{1,2} \w{1,6}|'+ # Multilaser
              'vibe k\d \w*|k\d \w*|' + # Lenovo
              'lumia \d{1,3}\w{1,2}|' + # Microsoft
              'quattro \w\d{1,3}|' + # Positivo
              'oneplus \d|' + # Oneplus
              'meizu \w\d\w|' + # Meizu
              'alcatel \w\d \w{2} max?') # Alcatel

res = {'MODELO': re_modelos,
       'QTD_MEMORIA': re_qtds_memoria,
       'QTD_ARMAZENAMENTO': re_qtds_armazenamento,}

In [39]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in birdie_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            # conserta regex de qtd memoria/armazenamento
            if (encontrado[0][-1] in [',', '.', ' ','\\','/'] and # comeca e termina com algum desses caracteres
                encontrado[0][0] in [',', '.', ' ','\\','/']):
                encontrados.append(encontrado[0][1:-1].strip())
            elif encontrado[0][-1] in [',', '.', ' ','\\','/']:  # termina com algum desses caracteres
                encontrados.append(encontrado[0][:-1].strip())
            elif encontrado[0][0] in [',', '.', ' ','\\','/']: # comeca com algum desses caracteres
                encontrados.append(encontrado[0][1:].strip())
            else:
                encontrados.append(encontrado[0].strip()) # nao comeca nem termina com algum desses caracteres
        else: 
            encontrados.append('')
        i += 1
    
    birdie_data[atributo] = encontrados
    birdie_data.reset_index(drop=True,inplace=True)

In [40]:
for atributo, regex in res.items():
    i = 0
    encontrados = []
    for smartphone in submarino_data['TITLE']:
        encontrado = re.findall(regex, smartphone)
        if encontrado:
            # conserta regex de qtd memoria/armazenamento
            if (encontrado[0][-1] in [',', '.', ' ','\\','/'] and # comeca e termina com algum desses caracteres
                encontrado[0][0] in [',', '.', ' ','\\','/']):
                encontrados.append(encontrado[0][1:-1].strip())
            elif encontrado[0][-1] in [',', '.', ' ','\\','/']:  # termina com algum desses caracteres
                encontrados.append(encontrado[0][:-1].strip())
            elif encontrado[0][0] in [',', '.', ' ','\\','/']: # comeca com algum desses caracteres
                encontrados.append(encontrado[0][1:].strip())
            else:
                encontrados.append(encontrado[0].strip()) # nao comeca nem termina com algum desses caracteres
        else: 
            encontrados.append('')
        i += 1
    
    submarino_data[atributo] = encontrados
    submarino_data.reset_index(drop=True,inplace=True)

In [41]:
# Casos sem MODELO
df = birdie_data[birdie_data['MODELO'] == '']

print('tamanho:', len(df))

df.head(5)

tamanho: 26


,TITLE,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
4,smartphone lg l90 dual d410 preto com tela de ...,,,
12,smartphone alcatel one touch pop c7 branco com...,,,
69,"smartphone samsung galaxy on7 g600fy 4g, dual,...",,,16gb
126,"celular alcatel pixi 4 - 5010e lite, dual chip...",,8gb,
127,celular smartphone tp-link neffos c5l 4g lte 4...,,8gb,


In [42]:
# Casos sem QTD_ARMAZENAMENTO e sem QTD_MEMORIA
df = birdie_data[(birdie_data['QTD_ARMAZENAMENTO'] == '') &
                 (birdie_data['QTD_MEMORIA'] == '' )]

print('tamanho:', len(df))

df.head(5)

tamanho: 122


,TITLE,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
1,smartphone samsung galaxy s4 mini duos preto c...,galaxy s4 mini,,
2,smartphone samsung galaxy s5 sm-g900m branco ...,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m dourado...,galaxy s5,,
4,smartphone lg l90 dual d410 preto com tela de ...,,,
5,smartphone samsung galaxy core plus branco co...,galaxy core plus,,


In [43]:
# Casos em que pelo menos algum spec está faltando
df = birdie_data[((birdie_data['QTD_ARMAZENAMENTO'] == '') |
                 (birdie_data['QTD_MEMORIA'] == '' ) |
                 (birdie_data['MODELO'] == '' ))]

print('tamanho:', len(df))

df.head()

tamanho: 420


,TITLE,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 preto co...,optimus l7,4gb,
1,smartphone samsung galaxy s4 mini duos preto c...,galaxy s4 mini,,
2,smartphone samsung galaxy s5 sm-g900m branco ...,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m dourado...,galaxy s5,,
4,smartphone lg l90 dual d410 preto com tela de ...,,,


In [44]:
# Casos em que nenhum specs esta presente
df = birdie_data[((birdie_data['QTD_ARMAZENAMENTO'] == '') &
                 (birdie_data['QTD_MEMORIA'] == '' ) &
                 (birdie_data['MODELO'] == '' ))]

print('tamanho:', len(df))

df.head()

tamanho: 8


,TITLE,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
4,smartphone lg l90 dual d410 preto com tela de ...,,,
12,smartphone alcatel one touch pop c7 branco com...,,,
152,motorola i867 ferrari,,,
155,celular j7 prime2 tv dourado + nota fiscal + l...,,,
190,samsung galaxy grand prime+ plus lacrado 4g me...,,,


In [45]:
# Casos em que todos os specs estao presentes
df = birdie_data[((birdie_data['QTD_ARMAZENAMENTO'] != '') &
                 (birdie_data['QTD_MEMORIA'] != '' ) &
                 (birdie_data['MODELO'] != '' ))]

print('tamanho:', len(df))

df.head()

tamanho: 168


,TITLE,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
35,smartphone lenovo vibe k6 plus grafite com 32g...,vibe k6 plus,3gb,32gb
36,smartphone lenovo vibe k6 plus dourado com 32g...,vibe k6 plus,3gb,32gb
37,smartphone samsung galaxy s7 edge black piano ...,galaxy s7 edge,4gb,128gb
44,smartphone asus zenfone 3 max zc553kl cinza co...,zenfone 3,3gb,32gb
48,smartphone asus zenfone 3 zoom ze553kl preto c...,zenfone 3,4gb,64gb


## 76% dos smartphones tem todos os specs. 1% não tem nenhum.  23% não tem pelo menos um.

In [46]:
# Spec mais importante:
print('smartphones sem modelo:', len(birdie_data['TITLE'][birdie_data['MODELO'] == '']))

smartphones sem modelo: 26


In [47]:
birdie_data['TITLE'][birdie_data['MODELO'] == ''].head(20)

# Dados com informação faltando/muito desordenados

4      smartphone lg l90 dual d410 preto com tela de ...
12     smartphone alcatel one touch pop c7 branco com...
69     smartphone samsung galaxy on7 g600fy 4g, dual,...
126    celular alcatel pixi 4 - 5010e lite, dual chip...
127    celular smartphone tp-link neffos c5l 4g lte 4...
152                                motorola i867 ferrari
155    celular j7 prime2 tv dourado + nota fiscal + l...
162    smartphone cat caterpillar s60 lte 32gb !! pro...
190    samsung galaxy grand prime+ plus lacrado 4g me...
193    iphone apple 8 64gb plus debloq s/juros leia a...
205    samsung galaxy k zoom c115 20mp 4g wifi tela d...
218    celular motorola g5 32gb android tela 5 câmera...
233                          nextel motorola i867 branco
269    iphone apple x 256gb orig grt apple leia anunc...
289    iphone apple x 64gb novo lacrado 5.8 garantia ...
302    samsung a7 2017 64gb novo a720 lacrado anatel ...
313    samsung a8 + plus a730 lançamento 64gb tela 6 ...
344    celular barato original 

In [48]:
print(birdie_data['MODELO'].unique())

['optimus l7' 'galaxy s4 mini' 'galaxy s5' '' 'galaxy core plus'
 'galaxy pocket 2 duos' 'galaxy ace 4 lite duos' 'g2 lite'
 'galaxy young 2 duos' 'galaxy win 2 duos' 'galaxy ace 4 duos' 'galaxy a7'
 'galaxy e5' 'galaxy a5' 'galaxy s6 edge' 'lumia 640xl' 'quattro x435'
 'k4' 'k10' 'ms50 colors' 'ms50 senior' 'zenfone ii' 'zenfone 2'
 'galaxy j2 prime' 'vibe k6 plus' 'galaxy s7 edge' 'zenfone 3' 'galaxy s8'
 'moto e4 plus' 'moto e4' 'moto c plus' 'moto z2' 'galaxy j7'
 'q6+ platinum' 'moto g5' 'zenfone 4' 'xperia xa1 g3116'
 'xperia xa1 plus g3426' 'xperia xz1 g8341' 'iphone 8' 'galaxy a8'
 'galaxy s9' 'moto g6' 'moto g6 plus' 'iphone 8 plus' 'moto e5' 'moto z3'
 'k9' '10 lite' 'galaxy s8+' 'galaxy j7 prime' 'galaxy s9+' 'iphone 5'
 'galaxy s3' 'iphone 6' 'cat s61' 'galaxy pocket plus' 'galaxy j5 prime'
 'moto g' 'mi a1' 'cat s60' 'xperia xz1' 'moto g4' 'moto c'
 'galaxy s5 mini' 'redmi note 4' 'iphone 4' 'moto g4 plus' 'galaxy j5'
 'mi s2' 'mi a2' 'mi 8' 'iphone 7' 'alcatel a3 xl max' 

In [49]:
print(birdie_data['QTD_MEMORIA'].unique())

['4gb' '' '8gb' '3gb' '2gb' '6gb']


In [50]:
print(birdie_data['QTD_ARMAZENAMENTO'].unique())

['' '64gb' '16gb' '32gb' '128gb' '256gb']


In [51]:
birdie_smartphones = []
smartphone = ''
for i in range(len(birdie_data)):
    for attribute in birdie_data.columns[1:]:
        spec = str(birdie_data[attribute][i])
        if spec == '':
            pass
        else:
            smartphone += spec + ' '
        
    smartphone = smartphone.strip()
    birdie_smartphones.append(smartphone)
    smartphone = ''

In [52]:
submarino_smartphones = []
smartphone = ''
for i in range(len(submarino_data)):
    for attribute in submarino_data.columns[1:]:
        spec = str(submarino_data[attribute][i])
        if spec == '':
            pass
        else:
            smartphone += spec + ' '
        
    smartphone = smartphone.strip()
    submarino_smartphones.append(smartphone)
    smartphone = ''

In [53]:
birdie_smartphones

['optimus l7 4gb',
 'galaxy s4 mini',
 'galaxy s5',
 'galaxy s5',
 '',
 'galaxy core plus',
 'galaxy pocket 2 duos',
 'galaxy s5',
 'galaxy ace 4 lite duos',
 'g2 lite',
 'galaxy young 2 duos',
 'galaxy young 2 duos',
 '',
 'galaxy win 2 duos',
 'galaxy ace 4 duos',
 'galaxy a7',
 'galaxy ace 4 duos',
 'galaxy e5',
 'galaxy a5',
 'galaxy s6 edge 64gb',
 'galaxy s6 edge 64gb',
 'galaxy s6 edge 64gb',
 'lumia 640xl',
 'galaxy win 2 duos',
 'galaxy win 2 duos',
 'quattro x435',
 'k4 8gb',
 'k10 16gb',
 'ms50 colors',
 'ms50 colors',
 'ms50 senior',
 'zenfone ii 16gb',
 'zenfone 2 16gb',
 'galaxy j2 prime 8gb',
 'galaxy j2 prime 8gb',
 'vibe k6 plus 3gb 32gb',
 'vibe k6 plus 3gb 32gb',
 'galaxy s7 edge 4gb 128gb',
 'k10 32gb',
 'k10 32gb',
 'k10 32gb',
 'k10 32gb',
 'k10 32gb',
 'k10 32gb',
 'zenfone 3 3gb 32gb',
 'galaxy s8 64gb',
 'galaxy s8 64gb',
 'galaxy s8 64gb',
 'zenfone 3 4gb 64gb',
 'moto e4 plus 2gb 16gb',
 'moto e4 2gb 16gb',
 'moto e4 2gb 16gb',
 'moto c plus 8gb',
 'moto e4 p

In [54]:
matches = []
i = 0
for smartphone in birdie_smartphones:
    match = process.extractOne(smartphone, submarino_smartphones, score_cutoff=100)
    matches.append(match)

In [55]:
indices = []
for match in matches:
    if match:
        indices.append(submarino_smartphones.index(match[0]))
    else:
        indices.append(None)

In [56]:
smartphones = []
modelos = []
qtds_memoria = []
qtds_armazenamento = []

for index in indices:
    if index:
        smartphone = submarino_data.iloc[index, 0]
        modelo = submarino_data.iloc[index, 1]
        qtd_memoria = submarino_data.iloc[index, 2]
        qtd_armazenamento = submarino_data.iloc[index, 3]
    else:
        smartphone = ''
        modelo = ''
        qtd_memoria = ''
        qtd_armazenamento = ''
        
    smartphones.append(smartphone)
    modelos.append(modelo)
    qtds_memoria.append(qtd_memoria)
    qtds_armazenamento.append(qtd_armazenamento)

In [57]:
resultados = pd.DataFrame(columns=['BIRDIE','SUBMARINO'])
resultados['BIRDIE'] = birdie_data.iloc[:, 0]
resultados['SUBMARINO'] = smartphones
resultados['MODELO'] = modelos
resultados['QTD_MEMORIA'] = qtds_memoria
resultados['QTD_ARMAZENAMENTO'] = qtds_armazenamento

In [58]:
resultados

,BIRDIE,SUBMARINO,MODELO,QTD_MEMORIA,QTD_ARMAZENAMENTO
0,smartphone lg optimus l7 ii dual p716 preto co...,smartphone lg optimus l7 ii desbloqueado tim a...,optimus l7,4gb,
1,smartphone samsung galaxy s4 mini duos preto c...,smartphone samsung galaxy s4 mini duos - tela ...,galaxy s4 mini,,
2,smartphone samsung galaxy s5 sm-g900m branco ...,smartphone samsung galaxy s5 - preto,galaxy s5,,
3,smartphone samsung galaxy s5 sm-g900m dourado...,smartphone samsung galaxy s5 - preto,galaxy s5,,
4,smartphone lg l90 dual d410 preto com tela de ...,,,,
5,smartphone samsung galaxy core plus branco co...,,,,
6,smartphone samsung galaxy pocket 2 duos sm-g11...,smartphone - samsung galaxy pocket 2 duos - br...,galaxy pocket 2 duos,,
7,smartphone samsung galaxy s5 duos sm-g900 pre...,smartphone samsung galaxy s5 - preto,galaxy s5,,
8,smartphone samsung galaxy ace 4 lite duos g313...,,,,
9,smartphone lg g2 lite d295 branco com tela de ...,smartphone lg g2 lite d295 dual preto/ titânio,g2 lite,,


In [59]:
resultados = resultados[resultados['SUBMARINO'] != '']

In [60]:
resultados.shape

(423, 5)

resultados.to_csv('data/matching.tsv', sep='\t', index=False)

# TODO: CHECAR AS REGEX (IPHONE, SAMSUNG, XIAOMI) NAO ESTA DETECTANDO IPHONE